# Unique features of Julia

In the last part of this introduction, we will dive a little deeper into what makes Julia unique. The core concepts we will treat here are:

- type system
- functions and methods
- multiple dispatch
- composability


## Links
- Stefan Karpinski, The unreasonable effectiveness of multiple dispatch - https://youtu.be/kc9HwsxE1OY?t=65

## Type system

Julia has the notion of abstract types and concrete types. Concrete types are the only ones that can actually have instances, and can only subtype abstract types, which is used to inherit behavior, not structure.

In [1]:
Base.show_supertypes(Float64)
println()
Base.show_supertypes(Int)

Float64 <: AbstractFloat <: Real <: Number <: Any
Int64 <: Signed <: Integer <: Real <: Number <: Any

In [2]:
# fine, but not very generic
struct Point1
    x::Float64
    y::Float64
end

# slow, because Julia cannot reason about the contents
struct Point2
    x::Any
    y  # nothing is the same as ::Any
end

# equally fast as Point1, but more generic
struct Point3{T}
    x::T
    y::T
end

In [3]:
p1 = Point1(1.1, -2.2)
isbits(p1)

true

In [4]:
p2 = Point2(1.1, -2.2)
isbits(p2)

false

In [5]:
p3 = Point3(1.1, -2.2)
@show isbits(p3)
p3

isbits(p3) = true


Point3{Float64}(1.1, -2.2)

In [6]:
Point3(1, -2)

Point3{Int64}(1, -2)

## Functions and methods

`+`, `abs` and `show` are examples of functions that are defined in the Julia Base library. A function can have many methods, which are functions that are defined for a specific set of input types. If you define your own type, it is common practice to add methods to existing functions, that tell julia how to apply a certain function to your types.

In [7]:
# show all the methods defined for the cos function
methods(abs)

# 13 methods for generic function "abs":
[1] abs(x::Unsigned) in Base at int.jl:179
[2] abs(x::Signed) in Base at int.jl:180
[3] abs(z::Complex) in Base at complex.jl:277
[4] abs(x::Rational) in Base at rational.jl:259
[5] abs(a::T) where T<:Dates.Period in Dates at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Dates/src/periods.jl:102
[6] abs(x::Bool) in Base at bool.jl:149
[7] abs(x::Float16) in Base at float.jl:522
[8] abs(x::Float32) in Base at float.jl:523
[9] abs(x::Float64) in Base at float.jl:524
[10] abs(x::Real) in Base at number.jl:164
[11] abs(::Missing) in Base at missing.jl:101
[12] abs(a::Pkg.Resolve.VersionWeight) in Pkg.Resolve at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Pkg/src/Resolve/versionweights.jl:32
[13] abs(a::Pkg.Resolve.FieldValue) in Pkg.Resolve at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Pkg/src/Resolve/fieldvalues.jl:51

In [8]:
# julia does not yet know how to take the absolute value of a point
abs(p3)

LoadError: MethodError: no method matching abs(::Point3{Float64})
[0mClosest candidates are:
[0m  abs([91m::Unsigned[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/int.jl:179
[0m  abs([91m::Signed[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/int.jl:180
[0m  abs([91m::Complex[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/complex.jl:277
[0m  ...

In [9]:
# let's teach it how
Base.abs(p::Point3) = Point3(abs(p.x), abs(p.y))

In [10]:
abs(p3)

Point3{Float64}(1.1, 2.2)

## Multiple dispatch

When a function is first called, Julia looks at the types of all input arguments, and will select the most specific method that applies. Then it will compile this method to native code, and then execute it. The second time that this method is called for the same types of input arguments, it will not have to compile again. Therefore you often see some latency on the first call to a method.

In cases where you care about performance, it is always good to check if the Julia compiler can figure out the types. If it cannot, it will still work, but will often be much slower since it needs to do more work at runtime versus compilation time. For an overview of performance tips, see also https://docs.julialang.org/en/v1/manual/performance-tips/index.html.

We can use macros to look into the different steps in the compilation process:

In [11]:
@code_typed abs(p3)

CodeInfo(
1 ─ %1 = Base.getfield(p, :x)::Float64
│   %2 = Base.abs_float(%1)::Float64
│   %3 = Base.getfield(p, :y)::Float64
│   %4 = Base.abs_float(%3)::Float64
│   %5 = %new(Point3{Float64}, %2, %4)::Point3{Float64}
└──      return %5
) => Point3{Float64}

In [12]:
@code_native abs(p3)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[9]:2 within `abs`
	movq	%rdi, %rax
; │ @ In[9]:2 within `abs` @ float.jl:524
	movups	(%rsi), %xmm0
	movabsq	$4775512416, %rcx               ## imm = 0x11CA48960
	andps	(%rcx), %xmm0
; │ @ In[9]:2 within `abs`
	movups	%xmm0, (%rdi)
	retq
	nopw	(%rax,%rax)
; └


Since we wrote our `Point3` type in a generic way with type parameter `T` that can represent any type, our type will now automatically work together with other coordinate types. We demonstrate that here using the Measurements package to represent uncertainty.

In [13]:
]add Measurements

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
    Updating `~/code/Project.toml`
  [eff96d63] + Measurements v2.7.2
    Updating `~/code/Manifest.toml`
  [49dc2e85] + Calculus v0.5.1
  [eff96d63] + Measurements v2.7.2
  [3cdcf5f2] + RecipesBase v1.2.1


In [14]:
using Measurements

In [15]:
# the plus minus ± symbol can be typed like \pm<tab>
uncertain_point = Point3(1.1 ± 0.2, -2.2 ± 0.4)

Point3{Measurement{Float64}}(1.1 ± 0.2, -2.2 ± 0.4)

In [16]:
abs(uncertain_point)

Point3{Measurement{Float64}}(1.1 ± 0.2, 2.2 ± 0.4)

## AbstractArrays

One nice example of how the julia community makes use of these unique features, can be seen in the different array types that are available:

- https://docs.julialang.org/en/v1/manual/interfaces/#man-interface-array-1
- https://github.com/JuliaArrays
- https://juliagpu.org/